In [1]:
import os
import tensorflow as tf
import numpy as np
import tvm
import tvm.relay as relay
import tvm.contrib.graph_runtime as runtime
from tvm.relay.expr_functor import ExprMutator
from tvm.contrib import util
from tvm.contrib.util import tempdir
import riptide.models
from riptide.get_models import get_model
from tvm import autotvm
from riptide.binary.binary_layers import Config, DQuantize, XQuantize

/home/jwfromm/anaconda3/envs/octoml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jwfromm/anaconda3/envs/octoml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jwfromm/anaconda3/envs/octoml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jwfromm/anaconda3/envs/octoml/lib/pyt

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [3]:
config = Config(actQ=DQuantize, weightQ=XQuantize, bits=1, use_act=False, use_bn=False, use_maxpool=True)
#config = Config(actQ=None, weightQ=None, bits=None, use_act=True, use_bn=True, use_maxpool=True)

In [4]:
with config:
    model = get_model('squeezenet')
#model = riptide.models.vggnet_normal.vggnet()
#model = tf.keras.models.Sequential(model.layers[:30])
#model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Conv2D(filters=96, strides=2, padding='same', kernel_size=7, use_bias=False, data_format='channels_last'))
#model.add(tf.keras.layers.BatchNormalization(center=False, scale=False))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
test_input = tf.keras.Input(shape=[224, 224, 3], batch_size=1, dtype='float32')
output = model(test_input)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
conv_ops = 0
glue_ops = 0
for i, layer in enumerate(model.layers):
    if 'conv2d' in layer.name:
        _, h, w, f = layer.output_shape
        k, _, c, _ = layer.weights[0].shape
        #print("conv: ", k*k*h*w*f*c)
        #print("glue: ", 14*h*w*f)
        #print("ratio: ", k*k*c / 14)
        conv_ops += k*k*h*w*f*c
        glue_ops += 14 * h*w*f

In [7]:
print(conv_ops)
print(glue_ops)
print(conv_ops.value / glue_ops)

786057216
51806720
15.172881355932203


In [8]:
mod, params = relay.frontend.from_keras(model, shape={'input_1': [1, 224, 224, 3]}, layout='NHWC')

In [9]:
seq = relay.transform.Sequential([relay.transform.FoldConstant(),
                                  relay.transform.EliminateCommonSubexpr()])
                                  #relay.transform.FuseOps(fuse_opt_level=2)])
mod = seq(mod)

In [10]:
target = tvm.target.arm_cpu("rasp3b")
target_host = 'llvm -device=arm_cpu -target=arm-linux-gnueabihf -mattr=+neon'
#target="llvm"

In [11]:
with relay.build_config(opt_level=3):
    graph, lib, params = relay.build(mod, target=target, params=params)
    #out = intrp.evaluate(func)(np.random.uniform(size=(1, 3, 28, 28)))

Cannot find config for target=llvm -device=arm_cpu -model=bcm2837 -target=armv7l-linux-gnueabihf -mattr=+neon, workload=('conv2d_nhwc_spatial_pack.arm_cpu', ('TENSOR', (1, 227, 227, 3), 'float32'), ('TENSOR', (7, 7, 3, 96), 'float32'), (4, 4), (0, 0, 0, 0), (1, 1), 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -device=arm_cpu -model=bcm2837 -target=armv7l-linux-gnueabihf -mattr=+neon, workload=('dense_nopack.x86', ('TENSOR', (1, 512), 'float32'), ('TENSOR', (1000, 512), 'float32'), None, 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -device=arm_cpu -model=bcm2837 -target=armv7l-linux-gnueabihf -mattr=+neon, workload=('bitserial_conv2d_nhwc.arm_cpu', ('TENSOR', (1, 14, 14, 64), 'int16'), ('TENSOR', (1, 1, 1, 8, 256), 'uint8'), (1, 1), (0, 0, 0, 0), 1, 1, 'uint8', 'int16', 1). A fallback configuration is used, which may bring gr

In [12]:
tmp = util.tempdir()
lib_fname = tmp.relpath('net.tar')
lib.export_library(lib_fname)

remote = autotvm.measure.request_remote(
    'rasp4b', 'tracker', 9191, timeout=10000)

In [13]:
# upload the library to remote device and load it
remote.upload(lib_fname)
rlib = remote.load_module('net.tar')

# create the remote runtime module
ctx = remote.cpu(0)
module = runtime.create(graph, rlib, ctx)
# set parameter (upload params to the remote device. This may take a while)
module.set_input(**params)

In [14]:
#module.set_input(0, np.random.uniform(size=(1, 3, 224, 224)))
module.set_input(0, np.random.uniform(size=(1, 224, 224, 3)))
module.run()
print(module.get_output(0).shape)

(1, 1000)


In [15]:
 # Evaluate
print("Evaluate inference time cost...")
ftimer = module.module.time_evaluator("run", ctx, number=10, repeat=1)
prof_res = np.array(ftimer().results) * 1000  # Convert to milliseconds
print("Mean inference time (std dev): %.2f ms (%.2f ms)" %
      (np.mean(prof_res), np.std(prof_res)))

Evaluate inference time cost...
Mean inference time (std dev): 68.64 ms (0.00 ms)
